In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import nltk
import numpy as np
import networkx as nx
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import subprocess  # For running PyInstaller
from tkinter import font

nltk.download('punkt')
nltk.download('stopwords')

def read_article(file_name):
    with open(file_name, "r", encoding='utf-8') as file:
        filedata = file.read()
    sentences = sent_tokenize(filedata)
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = set()

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def generate_summary(file_name, top_n=5):
    stop_words = set(stopwords.words('english'))
    summarize_text = []

    sentences = read_article(file_name)

    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    for i in range(top_n):
        summarize_text.append(ranked_sentence[i][1])

    return ". ".join(summarize_text)

def summarize_text():
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if file_path:
        summary = generate_summary(file_path)
        output_text.config(state="normal")
        output_text.delete(1.0, tk.END)
        output_text.insert(tk.END, summary)
        output_text.config(state="disabled")

def open_file():
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if file_path:
        subprocess.Popen(["notepad.exe", file_path])

def show_about():
    messagebox.showinfo("About", "Text Summarizer App\nVersion 2.0\n© 2023")

def save_summary():
    summary_text = output_text.get("1.0", "end-1c")
    if summary_text:
        save_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if save_path:
            with open(save_path, "w", encoding="utf-8") as file:
                file.write(summary_text)
            messagebox.showinfo("Saved", "Summary saved successfully!")

# Create the main application window
app = tk.Tk()
app.title("Text Summarizer")
app.geometry("800x600")  # Set window size

# Set colors and fonts
bg_color = "#E6E6E6"
button_color = "#4CAF50"
font_style = font.Font(family="Arial", size=12)

app.configure(bg=bg_color)

# Create buttons
file_button = tk.Button(app, text="Select File", command=summarize_text, bg=button_color, font=font_style)
file_button.pack(pady=10)

open_button = tk.Button(app, text="Open File", command=open_file, bg=button_color, font=font_style)
open_button.pack(pady=10)

about_button = tk.Button(app, text="About", command=show_about, bg=button_color, font=font_style)
about_button.pack(pady=10)

save_button = tk.Button(app, text="Save Summary", command=save_summary, bg=button_color, font=font_style)
save_button.pack(pady=10)

# Create a text widget to display the summary
output_text = tk.Text(app, wrap=tk.WORD, height=20, width=100)
output_text.config(state="disabled", bg="white", font=font_style)
output_text.pack()

# Start the GUI main loop
app.mainloop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rejuwan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rejuwan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
